<a href="https://colab.research.google.com/github/mahmoudmagdyhassan/NLP-and-CV/blob/main/Pneumonia_Classification_on_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!unzip /content/chest-xray-pneumonia.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths
train_path = '/content/chest_xray/train'
val_path = '/content/chest_xray/val'
test_path = '/content/chest_xray/test'

# Set image size and batch size
image_size = (180, 180)
batch_size = 32

# Create ImageDataGenerator instances
train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_data_gen
train_generator = train_data_gen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'  # Change to 'categorical' if you have multiple classes
)

# Flow validation images in batches using val_data_gen
val_generator = val_data_gen.flow_from_directory(
    val_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Flow test images in batches using test_data_gen
test_generator = test_data_gen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# Verify the class indices assigned by the generator
print("Class indices:", train_generator.class_indices)


In [ ]:
# Verify the class indices assigned by the generator
class_indices = train_generator.class_indices
print("Class indices:", class_indices)

# Get the number of samples for each class in the training set
num_normal_samples = sum(train_generator.classes == class_indices['NORMAL'])
num_pneumonia_samples = sum(train_generator.classes == class_indices['PNEUMONIA'])

print("Number of NORMAL samples:", num_normal_samples)
print("Number of PNEUMONIA samples:", num_pneumonia_samples)


In [ ]:
print("Image shape in training generator:", train_generator.image_shape)
print("Image shape in validation generator:", val_generator.image_shape)
print("Image shape in test generator:", test_generator.image_shape)


In [ ]:
import matplotlib.pyplot as plt

# Get a batch of images and labels from the training generator
images, labels = train_generator.next()

# Display the images along with their labels
plt.figure(figsize=(12, 8))
for i in range(min(36, batch_size)):
    plt.subplot(6, 6, i + 1)
    plt.imshow(images[i])
    plt.title(f"Class: {labels[i]}")
    plt.axis("off")

plt.show()


#Build the CNN


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def conv_block(filters, inputs):
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.MaxPool2D()(x)

    return outputs


def dense_block(units, dropout_rate, inputs):
    x = layers.Dense(units, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dropout(dropout_rate)(x)

    return outputs

In [ ]:
def build_model():
    inputs = keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
initial_bias = np.log([num_pneumonia_samples / num_normal_samples])
print("Initial bias: {:.5f}".format(initial_bias[0]))

TRAIN_IMG_COUNT = num_normal_samples + num_pneumonia_samples
weight_for_0 = (1 / num_normal_samples) * (TRAIN_IMG_COUNT) / 2.0
weight_for_1 = (1 / num_pneumonia_samples) * (TRAIN_IMG_COUNT) / 2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print("Weight for class 0: {:.2f}".format(weight_for_0))
print("Weight for class 1: {:.2f}".format(weight_for_1))


In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True)
initial_learning_rate = 0.015
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)


with strategy.scope():
    model = build_model()

    METRICS = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss="binary_crossentropy",
        metrics=METRICS,
    )

history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    class_weight=class_weight,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(["precision", "recall", "binary_accuracy", "loss"]):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history["val_" + met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model("/content/xray_model.h5")

# Load and preprocess the input image
img_path = "/content/chest_xray/test/NORMAL/IM-0005-0001.jpeg"
img = image.load_img(img_path, target_size=(180, 180))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize the pixel values to be between 0 and 1

# Make predictions
predictions = model.predict(img_array)

# Interpret the predictions
class_index = int(predictions[0] > 0.5)  # Use a threshold (e.g., 0.5) for binary classification
confidence = predictions[0][0] if class_index == 1 else 1 - predictions[0][0]  # Confidence score for the predicted class

# Print the results
print("Predicted class index:", class_index)
print("Predicted class confidence:", confidence)

# If you have class labels, you can map the index to the class label
class_labels = {0: 'NORMAL', 1: 'PNEUMONIA'}  # Replace with your own class labels
predicted_class = class_labels[class_index]
print("Predicted class:", predicted_class)


In [ ]:
def find_last_conv_layer(model):
    last_conv_layer = None

    for layer in reversed(model.layers):
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Conv3D)):
            last_conv_layer = layer
            break

    return last_conv_layer


# Find the last convolutional layer
last_conv_layer = find_last_conv_layer(model)

# Print information about the last convolutional layer
if last_conv_layer is not None:
    print("Last Convolutional Layer:")
    print("Layer Name:", last_conv_layer.name)
    print("Filters:", last_conv_layer.filters)
    print("Kernel Size:", last_conv_layer.kernel_size)
    print("Activation:", last_conv_layer.activation)
else:
    print("No Convolutional Layer found in the model.")

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(180, 180))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def get_gradcam_heatmap(model, img_array, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        class_channel = preds[:, np.argmax(preds[0])]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()

def overlay_heatmap(original_img, heatmap):
    heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlayed_img = cv2.addWeighted(original_img, 0.5, heatmap, 0.5, 0)
    return overlayed_img

# Load your trained model
model = tf.keras.models.load_model("/content/xray_model.h5")

# Load and preprocess the input image
img_path = "/content/chest_xray/test/NORMAL/IM-0005-0001.jpeg"
img_array = preprocess_image(img_path)

# Choose the last convolutional layer name in your model
last_conv_layer_name = "conv2d_5"

# Generate Grad-CAM heatmap
heatmap = get_gradcam_heatmap(model, img_array, last_conv_layer_name)

# Load the original image
original_img = cv2.imread(img_path)
original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

# Overlay the heatmap on the original image
overlayed_img = overlay_heatmap(original_img, heatmap)

# Display the images
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(original_img)
plt.title("Original Image")
plt.subplot(1, 2, 2)
plt.imshow(overlayed_img)
plt.title("Heatmap Overlay")
plt.show()
